In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 135.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="WNfIwAYjy3E5atGT2DPU")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(11)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Recognition-11 in yolov8:: 100%|██████████| 20262/20262 [00:03<00:00, 6367.65it/s]


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.4 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO
import shutil, os

In [7]:
import shutil, os

# Load pretrained YOLOv8 nano model
model = YOLO("yolov8n.pt")

# Train
results = model.train(
    data = dataset.location + "/data.yaml",   # use dataset path from Roboflow
    epochs = 10,
    imgsz = 640,
    batch = 16,
    workers = 2,
    device = 0
)

# Copy weights to a safe folder
os.makedirs("saved_models", exist_ok=True)
shutil.copy(
    "runs/detect/train2/weights/best.pt",
    "saved_models/license_plate_best.pt"
)
shutil.copy(
    "runs/detect/train2/weights/last.pt",
    "saved_models/license_plate_last.pt"
)

print("Weights saved in saved_models/")

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/License-Plate-Recognition-11/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

In [34]:
!pip install easyocr

In [60]:
#import libraries
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr
import re
from collections import defaultdict, deque


In [62]:
#import finetuned yolo and ocr
model = YOLO("saved_models/license_plate_best.pt")  # fine-tuned weights
reader = easyocr.Reader(['en'], gpu=True)

# Regex: 2 letters + 2 numbers + 3 letters
plate_pattern = re.compile(r"^[A-Z]{2}[0-9]{2}[A-Z]{3}$")

In [63]:
#correct plate format
def correct_plate_format(ocr_text):
    mapping_num_to_alpha = {"0": "O", "1": "I", "5": "S", "8": "B"}
    mapping_alpha_to_num = {"O": "0", "I": "1", "Z": "2", "S": "5", "B": "8"}

    ocr_text = ocr_text.upper().replace(" ", "")
    if len(ocr_text) != 7:
        return ""   # discard if wrong length

    corrected = []
    for i, ch in enumerate(ocr_text):
        if i < 2 or i >= 4:   # alphabet positions
            if ch.isdigit() and ch in mapping_num_to_alpha:
                corrected.append(mapping_num_to_alpha[ch])
            elif ch.isalpha():
                corrected.append(ch)
            else:
                return ""   # invalid char
        else:  # numeric positions
            if ch.isalpha() and ch in mapping_alpha_to_num:
                corrected.append(mapping_alpha_to_num[ch])
            elif ch.isdigit():
                corrected.append(ch)
            else:
                return ""   # invalid char

    return "".join(corrected)


In [103]:
import re

def recognize_plate(plate_crop):
    if plate_crop.size == 0:
        return ""

    # 1. Preprocessing (This part is already correct)
    gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.medianBlur(gray, 3)
    thresh = cv2.adaptiveThreshold(
        gray_blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 2
    )
    plate_resized = cv2.resize(thresh, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # 2. OCR (paragraph=False)
    try:
        ocr_result = reader.readtext(
            plate_resized,
            detail=0,
            allowlist='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789',
            paragraph=False  # This is very important
        )

        if len(ocr_result) < 1:   # If OCR didn’t read anything
            return ""

        # --- 3. NEW: “Junk” Filter (Trimming side garbage) ---

        # Clean every OCR block
        cleaned_blocks = []
        for block in ocr_result:
            clean_block = re.sub(r'[^A-Z0-9]', '', block.upper().replace(' ', ''))
            if clean_block:  # Keep only non-empty cleaned blocks
                cleaned_blocks.append(clean_block)

        if len(cleaned_blocks) == 0:
            return ""

        # Check 1: Is the first block junk?
        # (Junk = all letters AND length <= 2, like "E", "SK")
        if cleaned_blocks[0].isalpha() and len(cleaned_blocks[0]) <= 2:
            cleaned_blocks = cleaned_blocks[1:]   # Remove first block

        if len(cleaned_blocks) == 0:
            return ""

        # Check 2: Is the last block junk?
        # (Junk = all letters OR all numbers AND length <= 2, like "CK", "3", "4")
        if (cleaned_blocks[-1].isalpha() or cleaned_blocks[-1].isdigit()) and len(cleaned_blocks[-1]) <= 2:
            cleaned_blocks = cleaned_blocks[:-1]  # Remove last block

        if len(cleaned_blocks) == 0:
            return ""

        # --- 4. Join the real text ---
        # Now we have a junk-free list, e.g. ["6855", "1079"] or ["XZ8X", "8", "473"]
        cleaned_text = "".join(cleaned_blocks)   # → "68551079" or "XZ8X8473"

        # --- 5. Formatting and Correction ---

        # Case 1: "XZ8X8473" (length 8)
        if len(cleaned_text) == 8 and cleaned_text.startswith('X'):
            corrected = list(cleaned_text)
            if corrected[1] == '2': corrected[1] = 'Z'   # Convert "2" → "Z"
            if corrected[2] == 'B': corrected[2] = '8'   # Convert "B" → "8"
            cleaned_text = "".join(corrected)
            return f"{cleaned_text[:4]} {cleaned_text[4:]}"   # "XZ8X 8473"

        # Case 2: "68551079" (length 8, all digits)
        if len(cleaned_text) == 8 and cleaned_text.isdigit():
            return f"{cleaned_text[:4]} {cleaned_text[4:]}"   # "6855 1079"

        # Case 3: 7-digit plates (example: "XZ8X473" if '8' is missed)
        if len(cleaned_text) == 7 and cleaned_text.startswith('X'):
            return f"{cleaned_text[:4]} {cleaned_text[4:]}"   # "XZ8X 473"

        # If none of the above, but text exists, return as is
        if len(cleaned_text) >= 7:
            return cleaned_text

    except Exception as e:
        pass

    return ""   # Return empty string if filtering fails


In [104]:
#number plate stabilization
plate_history = defaultdict(lambda: deque(maxlen=10))  # last 10 predictions per box
plate_final = {}

# --- YEH RAHA NAYA "SMART" ID FUNCTION ---
def get_box_id(x1, y1, x2, y2):
    # Plate ka center point calculate karein
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2

    # Center point ko 50 pixel ke "grid" mein round karein
    # Isse ID "sticky" ban jaata hai aur plate hilne par bhi change nahi hota
    return f"{int(center_x / 50)}_{int(center_y / 50)}"

def get_stable_plate(box_id, new_text):
    if new_text:
        plate_history[box_id].append(new_text)
        # Majority vote
        most_common = max(set(plate_history[box_id]), key=plate_history[box_id].count)
        plate_final[box_id] = most_common
    return plate_final.get(box_id, "")

In [105]:
input_video = "vehicle_video.mp4"
output_video = "output_video_licenseplate.mp4"

cap = cv2.VideoCapture(input_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(
    output_video, fourcc,
    cap.get(cv2.CAP_PROP_FPS),
    (int(cap.get(3)), int(cap.get(4)))
)

CONF_THRESH = 0.3


In [106]:
# --- THIS IS THE CORRECT LOOP FOR GOOGLE COLAB ---

# Import tqdm for progress bar (if not already imported)
from tqdm import tqdm

# Setup frame count and progress bar
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
pbar = tqdm(total=frame_count, desc="Processing Video")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            conf = float(box.conf.cpu().numpy())
            if conf < CONF_THRESH:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy.cpu().numpy()[0])

            # Keep coordinates within frame boundary (Clipping)
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
            if x2 <= x1 or y2 <= y1:  # If box is invalid, skip it
                continue

            plate_crop = frame[y1:y2, x1:x2]

            # OCR with correction
            text = recognize_plate(plate_crop)

            # Stabilize text using history
            box_id = get_box_id(x1, y1, x2, y2)
            stable_text = get_stable_plate(box_id, text)

            # Draw a rectangle around the license plate
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

            # Overlay zoomed-in plate above detected region
            if plate_crop.size > 0:
                overlay_h, overlay_w = 150, 400
                plate_resized = cv2.resize(plate_crop, (overlay_w, overlay_h))

                oy1 = max(0, y1 - overlay_h - 40)
                ox1 = x1
                oy2, ox2 = oy1 + overlay_h, ox1 + overlay_w

                if oy2 <= frame.shape[0] and ox2 <= frame.shape[1]:
                    frame[oy1:oy2, ox1:ox2] = plate_resized

                # Display stabilized OCR text above overlay
                if stable_text:
                    cv2.putText(frame, stable_text, (ox1, oy1 - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 6)  # black outline
                    cv2.putText(frame, stable_text, (ox1, oy1 - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)  # white text

    # -------- WRITES THE PROCESSED FRAME TO OUTPUT VIDEO --------
    out.write(frame)
    pbar.update(1)  # Update progress bar

    # -------- THESE LINES ARE REMOVED FOR COLAB --------
    # cv2.imshow("Annotated Video", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Close progress bar after loop ends
pbar.close()

# Release files/resources
cap.release()
out.release()

print(f"Video processing complete! Saved to {output_video}")



Processing Video:   0%|          | 0/192 [00:00<?, ?it/s]Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

Processing Video: 100%|██████████| 192/192 [00:13<00:00, 14.04it/s]

Video processing complete! Saved to output_video_licenseplate.mp4
